In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten ,Activation
from keras.layers import Conv2D, MaxPooling2D
#from keras.preprocessing import image
from keras.utils import load_img, img_to_array
import os
import random


# Load the pre-trained AlexNet model
model = Sequential()
model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(256, (5,5), strides=(1,1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(384, (3,3), strides=(1,1), activation='relu', padding='same'))
model.add(Conv2D(384, (3,3), strides=(1,1), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), strides=(1,1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1000, activation='softmax'))
#model.load_weights('alexnet_weights.h5')
# Remove last layer to get feature extractor
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
#df = pd.read_csv('/content/drive/MyDrive/ALL_IMAGES.csv')
# Load and preprocess image one by one and extract features
model.summary()


In [ ]:
ID = pd.read_csv('/content/drive/MyDrive/ID.csv')
ID2=pd.read_csv('/content/drive/MyDrive/label_data.csv')

In [ ]:

features_matrix=np.zeros((6144,942,))
for i in range(len(ID)): # Change 10 with the number of images in your dataset
    file=[]
    folder_path = '/content/drive/MyDrive/ALL_IMAGES/'
    img_name=ID.ID[i]
    # List all the files in the folder
    #filenames = os.listdir(folder_path)

    # Define the first four characters of the filenames to search for
    prefix = str(img_name)

    # Filter the list of files based on the prefix
    files = [file for file in os.listdir(folder_path) if file.startswith(prefix)]

    # Randomly select a specified number of files from the filtered list
    num_files = len(files)
    selected_files = random.sample(files, num_files)
    img=[]
    features=[]
    # Print the paths to the selected image files
    for file in selected_files:
        print("Selected image file:", os.path.join(folder_path, file))
        img = image.load_img(os.path.join(folder_path, file), target_size=(227, 227))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        feature=feature_extractor.predict(img)
        features.append(feature)

    feature_vector=np.concatenate(features)
    F_vector = feature_vector.reshape(1, -1)



In [ ]:
#همراه با داده های کلینیکال
    features_matrix[:, i] = np.concatenate((F_vector.flatten(),ID2[i],np.zeros(features_matrix.shape[0]-(F_vector.shape[1]+ID2.shape[1]))))


In [ ]:
 #بدون دادهای کلینیکال
 features_matrix[:, i] = np.concatenate((F_vector.flatten(),np.zeros(features_matrix.shape[0]-(F_vector.shape[1]))))

In [ ]:
np.save('/content/drive/MyDrive/features_matrix2.npy', features_matrix)